In [1]:
!pip3 install xgboost

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 1.7 MB 1.6 MB/s            
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
from sklearn import datasets
import xgboost as xgb

In [3]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)

In [5]:
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

In [6]:
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

steps = 20  # The number of training iterations

In [7]:
model = xgb.train(param, D_train, steps)

In [8]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(D_test)
best_preds = np.asarray([np.argmax(line) for line in preds])

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

Precision = 0.9326923076923078
Recall = 0.9326923076923078
Accuracy = 0.9333333333333333


In [9]:
from sklearn.model_selection import GridSearchCV

clf = xgb.XGBClassifier()
parameters = {
     "eta"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
     "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
     "min_child_weight" : [ 1, 3, 5, 7 ],
     "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
     "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
     }

grid = GridSearchCV(clf,
                    parameters, n_jobs=4,
                    scoring="neg_log_loss",
                    cv=3)

grid.fit(X_train, Y_train)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_ca...
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                        

In [10]:
model.dump_model('dump.raw.txt')

In [11]:
!cat dump.raw.txt.raw.txtp.raw.txt

booster[0]:
0:[f2<2.5999999] yes=1,no=2,missing=1
	1:leaf=0.426589638
	2:leaf=-0.218769252
booster[1]:
0:[f2<2.5999999] yes=1,no=2,missing=1
	1:leaf=-0.213294834
	2:[f2<4.94999981] yes=3,no=4,missing=3
		3:[f3<1.60000002] yes=5,no=6,missing=5
			5:leaf=0.42281881
			6:leaf=-0.0620689802
		4:[f2<5.05000019] yes=7,no=8,missing=7
			7:leaf=-0.0360000096
			8:leaf=-0.21140942
booster[2]:
0:[f2<4.75] yes=1,no=2,missing=1
	1:leaf=-0.209708765
	2:[f2<4.94999981] yes=3,no=4,missing=3
		3:[f0<6.25] yes=5,no=6,missing=5
			5:leaf=0.128571421
			6:leaf=-7.66345476e-09
		4:leaf=0.409090936
booster[3]:
0:[f2<2.5999999] yes=1,no=2,missing=1
	1:leaf=0.294131458
	2:leaf=-0.195732966
booster[4]:
0:[f2<2.5999999] yes=1,no=2,missing=1
	1:leaf=-0.189599976
	2:[f3<1.54999995] yes=3,no=4,missing=3
		3:[f2<4.94999981] yes=5,no=6,missing=5
			5:leaf=0.292952508
			6:leaf=-0.1117642
		4:[f2<5.05000019] yes=7,no=8,missing=7
			7:leaf=-0.0383815542
			8:leaf=-0.187781304
booster[5]:
0:[f2<4.75] yes=1,no=2,missin